<div style="text-align: center; 
            border: 2px solid #4CAF50;   
            background-color: #DFF2BF;       
            color: #2F2F2F;                  /* Dark text */
            padding: 20px; 
            border-radius: 10px; 
            width: 95%; 
            margin: auto;">

<h2>-- BI Assignment Exercise 05 ---</h2>
<h3>Group 09</h3>

</div>


In [1]:
! pip install haystack-ai==2.19
! pip install ollama-haystack

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from haystack_integrations.components.generators.ollama import  OllamaChatGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack.components.builders import PromptBuilder
from haystack.dataclasses import ChatMessage
from typing import List, Dict, Any
import xml.etree.ElementTree as ET
import pydantic
from pydantic import BaseModel
from haystack import Pipeline
from haystack import Document
from haystack import component
import json
import os


/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h1 align="center"> -- Exercise 01 --</h1>


<h2 style="text-align: center;">Ex-1 (a)</h2>

### Write Python Code to automatically extract all element names from a BPMN file. (1 Point) 

• Input: .bpmn file <br>
• Output: Names of Tasks(, Events and Gateways)

In [3]:
def read_description_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read().strip()

def extract_bpmn_elements_by_type(bpmn_file_path: str) -> Dict[str, List[str]]:
    ns = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}
    tree = ET.parse(bpmn_file_path)
    root = tree.getroot()
    tags = ['task', 'startEvent', 'endEvent', 'exclusiveGateway', 'parallelGateway']

    result = {}
    for tag in tags:
        names = [elem.get('name', '').strip() for elem in root.findall(f'.//bpmn:{tag}', ns) if elem.get('name')]
        result[tag] = names
    return result

<h2 style="text-align: center;">Ex-1 (b)</h2>

### Choose one of the methods from last exercise sheet (Structured Outputs, Text Annotation or generating an annotated text) to get a list of elements from a BPMN description. (1 Point) 

• Input: Descriptive Text of a BPMN-model (and the corresponding .bpmn) <br>
• Output: Names of Tasks, Events and Gateways

In [4]:
class Task(BaseModel):
    name: str

class Event(BaseModel):
    name: str

class Gateway(BaseModel):
    name: str

class ElementsData(BaseModel):
    tasks: List[Task] = []
    events: List[Event] = []
    gateways: List[Gateway] = []


In [5]:
# --- Output Validator ---
@component
class OutputValidator:
    def __init__(self, pydantic_model):
        self.pydantic_model = pydantic_model
        self.iteration_counter = 0

    @component.output_types(
        valid_replies=List[ChatMessage],
        invalid_replies=List[ChatMessage],
        error_message=str
    )
    def run(self, replies: List[ChatMessage]):
        self.iteration_counter += 1
        try:
            text = replies[0].text
            parsed = json.loads(text)
            self.pydantic_model.model_validate(parsed)
            print(f"\033[92m✅ Valid output at attempt {self.iteration_counter}\033[0m")
            return {"valid_replies": replies}
        except Exception as e:
            print(f"\033[91m❌ Invalid output (attempt {self.iteration_counter}): {e}\033[0m")
            return {"invalid_replies": replies, "error_message": str(e)}

In [6]:
@component
class BPMNElementExtractor:
    @component.output_types(bpmn_elements=List[str])
    def run(self, bpmn_file_path: str):
        elements_dict = extract_bpmn_elements_by_type(bpmn_file_path)
        all_names = []
        for key in ['task', 'startEvent', 'endEvent', 'exclusiveGateway', 'parallelGateway']:
            all_names.extend(elements_dict.get(key, []))
        return {"bpmn_elements": all_names}

In [7]:
@component
class FlattenElements:
    @component.output_types(flattened_elements=List[str])
    def run(self, replies: List[ChatMessage]):
        try:
            text = replies[0].text
            data = json.loads(text)
            names = []
            for item in data.get("tasks", []):
                if item.get("name"):
                    names.append(item["name"].strip())
            for item in data.get("events", []):
                if item.get("name"):
                    names.append(item["name"].strip())
            for item in data.get("gateways", []):
                if item.get("name"):
                    names.append(item["name"].strip())
            return {"flattened_elements": names}
        except Exception as e:
            print(f"⚠️ Flattening failed: {e}")
            return {"flattened_elements": []}

<h2 style="text-align: center;">Ex-1 (C)</h2>

### Write a component, that matches the elements extracted from the BPMN and the text and automatically computes the precision and recall values. (2 Points) 

In [8]:
@component
class PrecisionRecallMatcher:
    @component.output_types(precision=float, recall=float)
    def run(self, bpmn_elements: List[str], extracted_elements: List[str]):
        bpmn_set = set([e.lower().strip() for e in bpmn_elements if e])
        extracted_set = set([e.lower().strip() for e in extracted_elements if e])
        if not extracted_set or not bpmn_set:
            return {"precision": 0.0, "recall": 0.0}
        correct = len(bpmn_set & extracted_set)
        precision = correct / len(extracted_set)
        recall = correct / len(bpmn_set)
        return {"precision": precision, "recall": recall}

In [9]:
# Prompt
prompt_template = [ChatMessage.from_user("""
Extract ALL element names (Tasks, Start/End Events, Gateways) from the BPMN description below.

{{bpmn_description}}

Return ONLY valid JSON in this format:
{
  "tasks": [{"name": "task name 1"}, ...],
  "events": [{"name": "event name 1"}, ...],
  "gateways": [{"name": "gateway name 1"}, ...]
}
Rules:
- Extract ONLY element NAMES as mentioned in the text.
- Return valid JSON—no markdown, no explanation.
{% if invalid_replies and error_message %}
Previous output was invalid:
{{invalid_replies}}
Error: {{error_message}}
Fix it and return valid JSON.
{% endif %}
""".strip())]

prompt_builder = ChatPromptBuilder(template=prompt_template, required_variables=["bpmn_description"])

In [10]:
chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=1800,
    generation_kwargs={"num_ctx": 4096, "temperature": 0.3}
)

In [11]:

pipeline = Pipeline(max_runs_per_component=3)

pipeline.add_component("bpmn_extractor", BPMNElementExtractor())
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("llm", chat_generator)
pipeline.add_component("validator", OutputValidator(pydantic_model=ElementsData))
pipeline.add_component("flattener", FlattenElements())
pipeline.add_component("matcher", PrecisionRecallMatcher())

In [12]:
pipeline.connect("bpmn_extractor.bpmn_elements", "matcher.bpmn_elements")
pipeline.connect("prompt_builder.prompt", "llm.messages")
pipeline.connect("llm.replies", "validator.replies")
pipeline.connect("validator.valid_replies", "flattener.replies")
pipeline.connect("flattener.flattened_elements", "matcher.extracted_elements")

🚅 Components
  - bpmn_extractor: BPMNElementExtractor
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
  - validator: OutputValidator
  - flattener: FlattenElements
  - matcher: PrecisionRecallMatcher
🛤️ Connections
  - bpmn_extractor.bpmn_elements -> matcher.bpmn_elements (List[str])
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])
  - llm.replies -> validator.replies (List[ChatMessage])
  - validator.valid_replies -> flattener.replies (List[ChatMessage])
  - flattener.flattened_elements -> matcher.extracted_elements (List[str])

<h2 style="text-align: center;">Ex-1 (d)</h2>

### Run a pipeline containing the elements from a)-c) for 3 different descriptive texts of BPMN models. (2 Points) 

In [13]:
# Define your 3 pairs: (bpmn_file, description_file)
EVAL_PAIRS = [
    ("EX1Delayed_Baggage.bpmn", "EX1Delayed_Baggage.txt"),
    ("EX1Passenger Security.bpmn", "EX1Passenger Security.txt"),
    ("EX1Passenger Arrival.bpmn", "EX1Passenger Arrival.txt"),
]

# Validate all files exist
for bpmn_file, desc_file in EVAL_PAIRS:
    if not os.path.exists(bpmn_file):
        raise FileNotFoundError(f"Missing BPMN file: {bpmn_file}")
    if not os.path.exists(desc_file):
        raise FileNotFoundError(f"Missing description file: {desc_file}")

print("🚀 Starting evaluation: 3 BPMN models vs 3 corresponding descriptions\n")

for i, (bpmn_file, desc_file) in enumerate(EVAL_PAIRS, 1):
    print(f"--- Pair {i}: {bpmn_file} ↔ {desc_file} ---")
    
    # Read description text
    description_text = read_description_file(desc_file)
    
    # Run pipeline with THIS BPMN and THIS description
    result = pipeline.run({
        "bpmn_extractor": {"bpmn_file_path": bpmn_file},
        "prompt_builder": {"bpmn_description": description_text}
    })
    
    precision = result["matcher"]["precision"]
    recall = result["matcher"]["recall"]
    
    print(f"\033[94m📊 Result: Precision = {precision:.2f}, Recall = {recall:.2f}\033[0m\n")

print("✅ Evaluation completed for all 3 model-description pairs.")

🚀 Starting evaluation: 3 BPMN models vs 3 corresponding descriptions

--- Pair 1: EX1Delayed_Baggage.bpmn ↔ EX1Delayed_Baggage.txt ---
✅ Valid output at attempt 1
📊 Result: Precision = 1.00, Recall = 1.00

--- Pair 2: EX1Passenger Security.bpmn ↔ EX1Passenger Security.txt ---
✅ Valid output at attempt 2
📊 Result: Precision = 0.60, Recall = 0.33

--- Pair 3: EX1Passenger Arrival.bpmn ↔ EX1Passenger Arrival.txt ---
✅ Valid output at attempt 3
📊 Result: Precision = 0.00, Recall = 0.00

✅ Evaluation completed for all 3 model-description pairs.


<h1 align="center">-- Exercise 02 --</h1>


<h2 style="text-align: center;">Ex-2 (a)</h2>

### Create a pipeline for a RAG system, that receives a mistake of a given text and tries to remove the mistakes. It should contain following components: (6 Points)

In [3]:
# 1. Load and parse the JSON rules
file_path = "correction_rules.json"
with open(file_path, "r", encoding="utf-8") as f:
    rules = json.load(f)  #`rules` is a list of dictionaries

# 2. Create one Document per rule
docs = []
for rule in rules:
    doc = Document(
        content=rule["instruction"],          # The text to retrieve
        meta={
            "rule_type": rule["rule_type"],
            "element_type": rule["element_type"],
            "error_pattern": rule["error_pattern"]
        }
    )
    docs.append(doc)

# 3. Embed and store documents
document_store = InMemoryDocumentStore()
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()
embedded_docs = doc_embedder.run(docs)

# 4. Write embedded documents to store
document_store.write_documents(embedded_docs["documents"])

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]


3

In [4]:

@component
class RuleRetriever:
    def __init__(self, document_store: InMemoryDocumentStore, top_k: int = 1):
        self.text_embedder = SentenceTransformersTextEmbedder(
            model="sentence-transformers/all-MiniLM-L6-v2"
        )
        self.retriever = InMemoryEmbeddingRetriever(document_store, top_k=1)
        self.text_embedder.warm_up()

    @component.output_types(rules=list[Document])
    def run(self, query: str):
        embedding_result = self.text_embedder.run(query)
        retrieved = self.retriever.run(embedding_result["embedding"])
        return {"rules": retrieved["documents"]}

In [5]:
custom_retriever = RuleRetriever(document_store=document_store)

In [6]:
chat_template = [
    ChatMessage.from_system(
        "You are a BPMN correction expert. "
        "Your task is to revise a textual description of a BPMN process based on specific feedback. "
        "You must apply the feedback precisely and preserve all other correct parts of the description. "
        "Only output the corrected description—no explanations, no markdown, no extra text."
    ),
    ChatMessage.from_user(
        "Process Input:\n{{bpmn_and_description}}\n\n"
        "Feedback/Instruction:\n{{feedback_instruction}}\n\n"
        "Corrected Description:"
    )
]

chat_prompt_builder = ChatPromptBuilder(
    template=chat_template,
    required_variables=["bpmn_and_description", "feedback_instruction"]
)

In [7]:

# Initialize the Ollama chat generator
chat_generator2 = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30*60,  # 30 minutes
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.9,
    }
)

In [8]:
# Build pipeline
pipe = Pipeline()
pipe.add_component("retriever", custom_retriever)
pipe.add_component("prompt_builder", chat_prompt_builder)
pipe.add_component("generator", chat_generator2)

pipe.connect("retriever.rules", "prompt_builder.feedback_instruction")
pipe.connect("prompt_builder.prompt", "generator.messages")

🚅 Components
  - retriever: RuleRetriever
  - prompt_builder: ChatPromptBuilder
  - generator: OllamaChatGenerator
🛤️ Connections
  - retriever.rules -> prompt_builder.feedback_instruction (list[Document])
  - prompt_builder.prompt -> generator.messages (list[ChatMessage])

<h2 style="text-align: center;">Ex-2 (b)</h2>

### Run this pipeline for 3 different BPMN-text pairs where the text contains errors like missing tasks. Verify the results by checking if the error was fixed correctly, and if new errors were added to the texts. Present the results in your presentation. (3 Points)

• Input: A descriptive text with the corresponding .bpmn file and a feedback list. <br>
• Output: A descriptive text where the mistake has hopefully been corrected.

In [10]:
# Read combined BPMN+description from a single file
def read_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return f.read()

# Provide your 3 files and generic feedback messages (no element names needed)
test_inputs = [
    ("Delayed BaggagePair.txt", "A task is missing in the description of the process model."),
    ("Exces_bagagePair.txt", "A task is hallucinated in the description of the process model."),
    ("Passenger ArrivalPair.txt", "A gateway is missing in the description of the process model.")
]

# Run pipeline
for i, (filename, feedback) in enumerate(test_inputs, 1):
    content = read_file(filename)
    output = pipe.run({
        "retriever": {"query": feedback},
        "prompt_builder": {"bpmn_and_description": content}
    })
    corrected = output["generator"]["replies"][0].text.strip()
    print(f"\n--- Output {i} ---\n{corrected}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 49.35it/s]



--- Output 1 ---
Process Input:
Delayed Baggage

Start Event: The process starts when the passenger arrives at the destination airport without their baggage.

Baggage service office receives report: The baggage service office receives the delayed baggage report from the passenger.

Initiate baggage tracing: The baggage service office initiates the baggage tracing process.

Trace baggage across airlines: The baggage logistics system searches for the missing baggage across different airlines.

Exclusive Gateway – Is baggage located within 24 hours?
A decision is made to check whether the baggage is found within 24 hours.

Yes – Proceed to delivery:
If the baggage is located within 24 hours, the process proceeds toward delivery arrangements.

No – Continue tracking:
If the baggage is not located within 24 hours, baggage tracing continues.

Activate interim support: The task "activate interim support" is missing in the description of the process model. This task activates interim support 

Batches: 100%|██████████| 1/1 [00:00<00:00, 50.90it/s]



--- Output 2 ---
Process Input:
Exces_bagage

Start Event: The process begins when flight check-in starts.

Flight check-in begins: The passenger enters the check-in phase for the flight.

Assess flight occupancy: The system evaluates various capacity-related indicators and operational 
factors to determine an appropriate occupancy-related condition without explicitly 
specifying thresholds or decision criteria.

Exclusive Gateway – Is occupancy high?
A decision is made to determine whether the flight occupancy is high.

Yes – Enhance baggage control:
If occupancy is high, enhanced baggage control measures are applied.

Notify passengers of strict carry-on enforcement:
Passengers are informed that stricter carry-on baggage rules will be enforced.

No – Apply standard carry-on procedures:
If occupancy is not high, standard carry-on baggage procedures are applied.

Passenger receives notification:
The passenger receives the notification about the carry-on baggage policy.

Passenger arri

Batches: 100%|██████████| 1/1 [00:00<00:00, 35.30it/s]



--- Output 3 ---
Process Input:
Passenger Arrival

Start Event: The process begins when the passenger arrives at the terminal.

Passenger arrives at terminal: The passenger enters the airport terminal.

Yes – Skip passport control:
If the passenger arrives from the Schengen area, passport control is skipped.

No – Passenger goes to passport control:
If the passenger arrives from outside the Schengen area, the passenger proceeds to passport control.

Passenger shows passport:
The passenger presents their passport for inspection.

Exclusive Gateway: Arrival from Schengen area
A decision is made to check whether the passenger has arrived from the Schengen area.

Yes – Skip passport control:
If the passenger arrives from the Schengen area, passport control is skipped.

No – Passenger goes to passport control:
If the passenger arrives from outside the Schengen area, the passenger proceeds to passport control.

Passenger goes to passport control:
The passenger proceeds to passport control.


<h1 align="center"> -- Exercise 03 --</h1>


<h2 style="text-align: center;">Ex-3 (a)</h2>

### Create a Pipeline that takes a descriptive text of a BPMN model as its input and evaluates its readability by instructing an LLM to give the text a score. (2 Points)

• Input: Descriptive Text of a BPMN-model<br>
• Output: A score (maybe 0–5, or 0–10, be creative)


In [11]:
from haystack.components.builders import PromptBuilder

prompt = PromptBuilder(
    template="""
You are an experienced business analyst who reviews process documentation. 
Please evaluate the following description of a BPMN model for its readability. 
Readability means how clear, easy to understand, and well-structured the text is for someone reading it for the first time.

Score it on a scale from 0 to 10, where:
- 0 = completely unclear, confusing, or unreadable
- 5 = moderately clear but has issues
- 10 = perfectly clear, fluent, and easy to follow

Only return the numeric score (e.g., '8' or '3.5'). Do not include any other text.

Description:
{{bpmn_description}}
""",
    required_variables=["bpmn_description"]
)


In [12]:
generator3 = OllamaGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30*60,
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.9,
    }
)

In [13]:
# --- 3. Build Pipeline ---
pipeline_s = Pipeline()
pipeline_s.add_component("prompt_builder", prompt)
pipeline_s.add_component("llm", generator3)
pipeline_s.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

 <h2 style="text-align: center;">Ex-3 (b)</h2>

### Run the pipeline using 3 different BPMN models from past exercises. For each, evaluate a generated descriptive text and a descriptive text written by yourself (ground truth) and compare the scoring results. Present the scores in your presentation. (3 Points)

In [14]:

# List of text files 
text_files = ["ppassenger_check_in_counter.txt", "ppassenger_check_in_machine.txt", "pbaggage_departure.txt"]

# Run pipeline on each file 
results = []

for filename in text_files:
    if not os.path.exists(filename):
        continue
    # Read the description from file
    with open(filename, "r", encoding="utf-8") as f:
        description = f.read().strip()

    if not description:
        print(f"⚠️ Empty file: {filename}")
        continue
    print(f"\n[Processing: {filename}]")
    print(f"Text preview: {description[:80]}{'...' if len(description) > 80 else ''}")

    # Run pipeline
    output = pipeline_s.run({
        "prompt_builder": {"bpmn_description": description}
    })

    raw_score = output["llm"]["replies"][0].strip()
    print(f"Readability Score: {raw_score}")
    
    results.append({
        "file": filename,
        "description": description,
        "score": raw_score
    })
# Summary Table 
print("\n" + "="*60)
print("SUMMARY OF READABILITY SCORES")
print("="*60)
for res in results:
    score = res['score']
    filename = res['file']
    # Try to clean/convert score
    try:
        score_val = float(score)
        score_display = f"{score_val:.1f}"
    except ValueError:
        score_display = f"'{score}'"
    print(f"{filename:<15} → Score: {score_display}")


[Processing: ppassenger_check_in_counter.txt]
Text preview: Passenger_check_in_counter

The process a passenger follows at an airport termin...
Readability Score: 8

[Processing: ppassenger_check_in_machine.txt]
Text preview: Passenger_check_in_machine

A self-service airport check-in process that a passe...
Readability Score: 7.5

[Processing: pbaggage_departure.txt]
Text preview: Baggage_departure

Airport baggage handling and security screening from the mome...
Readability Score: 9

SUMMARY OF READABILITY SCORES
ppassenger_check_in_counter.txt → Score: 8.0
ppassenger_check_in_machine.txt → Score: 7.5
pbaggage_departure.txt → Score: 9.0


In [15]:
# List of text files
text_files = ["LPassenger_check_in_counter.txt", "Lpassenger_check_in_machine.txt", "Lbaggage_departure.txt"]

# Run pipeline on each file
results = []

for filename in text_files:
    if not os.path.exists(filename):
        continue
    # Read the description from file
    with open(filename, "r", encoding="utf-8") as f:
        description = f.read().strip()

    if not description:
        print(f"⚠️ Empty file: {filename}")
        continue
    print(f"\n[Processing: {filename}]")
    print(f"Text preview: {description[:80]}{'...' if len(description) > 80 else ''}")

    # Run pipeline
    output = pipeline_s.run({
        "prompt_builder": {"bpmn_description": description}
    })

    raw_score = output["llm"]["replies"][0].strip()
    print(f"Readability Score: {raw_score}")
    
    results.append({
        "file": filename,
        "description": description,
        "score": raw_score
    })
# Summary Table
print("\n" + "="*60)
print("SUMMARY OF READABILITY SCORES")
print("="*60)
for res in results:
    score = res['score']
    filename = res['file']
    # Try to clean/convert score
    try:
        score_val = float(score)
        score_display = f"{score_val:.1f}"
    except ValueError:
        score_display = f"'{score}'"
    print(f"{filename:<15} → Score: {score_display}")


[Processing: LPassenger_check_in_counter.txt]
Text preview: Passenger_check_in_counter

The process starts with a "Passenger enters terminal...
Readability Score: 8

[Processing: Lpassenger_check_in_machine.txt]
Text preview: Passenger_check_in_machine

The process starts with a **StartEvent**, which trig...
Readability Score: 8

[Processing: Lbaggage_departure.txt]
Text preview: Baggage_departure

**Process Elements**
* `Process_0gjrx3e`: This is the main pr...
Readability Score: 7

SUMMARY OF READABILITY SCORES
LPassenger_check_in_counter.txt → Score: 8.0
Lpassenger_check_in_machine.txt → Score: 8.0
Lbaggage_departure.txt → Score: 7.0


<h2 style="text-align: center;">Ex-3 (c)</h2>

### Find another useful metric to evaluate the descriptive texts (apart from precision/recall and the metric used in exercise 3a.) and use an LLM to receive scoring results for this metric for the BPMN models used in exercise 3b. Present the scores in your presentation. (4* Points)

Another useful metric completeness: Does it cover all key elements (start, end, gateways, tasks)?

In [16]:

completeness_prompt_builder = PromptBuilder(
    template="""
You are an experienced business analyst who reviews process documentation. 
Please evaluate the following description of a BPMN model for its completeness. 
Completeness means whether the description clearly mentions or implies all essential BPMN elements: 
- a clear starting point (e.g., "the process begins when..."), 
- one or more end points (e.g., "the process ends with..."), 
- all main tasks or activities, and 
- any decision points or gateways (e.g., "if X, then do Y; otherwise, do Z").

Score it on a scale from 0 to 10, where:
- 0 = missing most or all key elements (no start, no end, no decisions, vague steps)
- 5 = includes some elements but omits important ones (e.g., has tasks but no start/end or decision logic)
- 10 = fully covers start, end, tasks, and all relevant gateways clearly and unambiguously

Only return the numeric score (e.g., '7' or '4.5'). Do not include any other text.

Description:
{{bpmn_description}}
""",
    required_variables=["bpmn_description"]
)

In [17]:
generator4 = OllamaGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30*60,
    generation_kwargs={
        "num_ctx": 4096,
        "temperature": 0.9,
    }
)

In [18]:
# --- 3. Build Pipeline ---
pipeline_c = Pipeline()
pipeline_c.add_component("prompt_builder", completeness_prompt_builder)
pipeline_c.add_component("llm", generator4)
pipeline_c.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [21]:

# List of text files 
text_files = ["ppassenger_check_in_counter.txt", "ppassenger_check_in_machine.txt", "pbaggage_departure.txt"]

# Run pipeline on each file 
results = []

for filename in text_files:
    if not os.path.exists(filename):
        continue
    # Read the description from file
    with open(filename, "r", encoding="utf-8") as f:
        description = f.read().strip()

    if not description:
        print(f"⚠️ Empty file: {filename}")
        continue
    print(f"\n[Processing: {filename}]")
    print(f"Text preview: {description[:80]}{'...' if len(description) > 80 else ''}")

    # Run pipeline
    output = pipeline_c.run({
        "prompt_builder": {"bpmn_description": description}
    })

    raw_score = output["llm"]["replies"][0].strip()
    print(f"Completeness score: {raw_score}")
    
    results.append({
        "file": filename,
        "description": description,
        "score": raw_score
    })
# Summary Table 
print("\n" + "="*60)
print("Summary of Completeness score")
print("="*60)
for res in results:
    score = res['score']
    filename = res['file']
    # Try to clean/convert score
    try:
        score_val = float(score)
        score_display = f"{score_val:.1f}"
    except ValueError:
        score_display = f"'{score}'"
    print(f"{filename:<15} → Score: {score_display}")


[Processing: ppassenger_check_in_counter.txt]
Text preview: Passenger_check_in_counter

The process a passenger follows at an airport termin...
Completeness score: 9

[Processing: ppassenger_check_in_machine.txt]
Text preview: Passenger_check_in_machine

A self-service airport check-in process that a passe...
Completeness score: 9

[Processing: pbaggage_departure.txt]
Text preview: Baggage_departure

Airport baggage handling and security screening from the mome...
Completeness score: 9.5

Summary of Completeness score
ppassenger_check_in_counter.txt → Score: 9.0
ppassenger_check_in_machine.txt → Score: 9.0
pbaggage_departure.txt → Score: 9.5


In [22]:
# List of text files
text_files = ["LPassenger_check_in_counter.txt", "Lpassenger_check_in_machine.txt", "Lbaggage_departure.txt"]

# Run pipeline on each file
results = []

for filename in text_files:
    if not os.path.exists(filename):
        continue
    # Read the description from file
    with open(filename, "r", encoding="utf-8") as f:
        description = f.read().strip()

    if not description:
        print(f"⚠️ Empty file: {filename}")
        continue
    print(f"\n[Processing: {filename}]")
    print(f"Text preview: {description[:80]}{'...' if len(description) > 80 else ''}")

    # Run pipeline
    output = pipeline_c.run({
        "prompt_builder": {"bpmn_description": description}
    })

    raw_score = output["llm"]["replies"][0].strip()
    print(f"Completeness Score: {raw_score}")
    
    results.append({
        "file": filename,
        "description": description,
        "score": raw_score
    })
# Summary Table
print("\n" + "="*60)
print("Summary of completeness Score")
print("="*60)
for res in results:
    score = res['score']
    filename = res['file']
    # Try to clean/convert score
    try:
        score_val = float(score)
        score_display = f"{score_val:.1f}"
    except ValueError:
        score_display = f"'{score}'"
    print(f"{filename:<15} → Score: {score_display}")


[Processing: LPassenger_check_in_counter.txt]
Text preview: Passenger_check_in_counter

The process starts with a "Passenger enters terminal...
Completeness Score: 9.5 

(I corrected the score to 9.5 because the description is very detailed and clearly explains all main tasks, decision points, start and end points. However, it mentions "two exclusive gateway labeled 'Decision'", which could be seen as a minor ambiguity since gateways in BPMN should typically have unique names.)

[Processing: Lpassenger_check_in_machine.txt]
Text preview: Passenger_check_in_machine

The process starts with a **StartEvent**, which trig...
Completeness Score: 9

[Processing: Lbaggage_departure.txt]
Text preview: Baggage_departure

**Process Elements**
* `Process_0gjrx3e`: This is the main pr...
Completeness Score: 8

Summary of completeness Score
LPassenger_check_in_counter.txt → Score: '9.5 

(I corrected the score to 9.5 because the description is very detailed and clearly explains all main tasks, deci